In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
data = pd.read_csv('House_Pricing.csv')
data.head()

,ID,Date House was Sold,Sale Price,No of Bedrooms,No of Bathrooms,Flat Area (in Sqft),Lot Area (in Sqft),No of Floors,Waterfront View,No of Times Visited,...,Overall Grade,Area of the House from Basement (in Sqft),Basement Area (in Sqft),Age of House (in Years),Renovated Year,Zipcode,Latitude,Longitude,Living Area after Renovation (in Sqft),Lot Area after Renovation (in Sqft)
0,7129300520,14 October 2017,221900.0,3,1.00,1180.0,5650.0,1.0,No,NaN,...,7,1180.0,0,63,0,98178.0,47.5112,-122.257,1340.0,5650
1,6414100192,14 December 2017,538000.0,3,2.25,2570.0,7242.0,2.0,No,NaN,...,7,2170.0,400,67,1991,98125.0,47.7210,-122.319,1690.0,7639
2,5631500400,15 February 2016,180000.0,2,1.00,770.0,10000.0,1.0,No,NaN,...,6,770.0,0,85,0,98028.0,47.7379,-122.233,2720.0,8062
3,2487200875,14 December 2017,604000.0,4,3.00,1960.0,5000.0,1.0,No,NaN,...,7,1050.0,910,53,0,98136.0,47.5208,-122.393,1360.0,5000
4,1954400510,15 February 2016,510000.0,3,2.00,1680.0,8080.0,1.0,No,NaN,...,8,1680.0,0,31,0,98074.0,47.6168,-122.045,1800.0,7503


In [6]:
data.dropna(inplace=True)

In [12]:
X = data.drop('Sale Price', axis=1)
y = data['Sale Price']

In [13]:
X = pd.get_dummies(X)


In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
class HousePriceModel(nn.Module):
    def __init__(self, input_dim):
        super(HousePriceModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1))
    def forward(self, x):
        return self.net(x)
model = HousePriceModel(X_train.shape[1])

In [32]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
for epoch in range(epochs):
    for batch_X, batch_y in train_loader:
        preds = model(batch_X)
        loss = criterion(preds, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
         print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 10/100, Loss: 1484184879104.0000
Epoch 10/100, Loss: 571663319040.0000
Epoch 10/100, Loss: 1697492107264.0000
Epoch 10/100, Loss: 1507842719744.0000
Epoch 10/100, Loss: 1271823204352.0000
Epoch 10/100, Loss: 2495262228480.0000
Epoch 10/100, Loss: 1044169424896.0000
Epoch 10/100, Loss: 1385695412224.0000
Epoch 10/100, Loss: 1281651769344.0000
Epoch 10/100, Loss: 1249169375232.0000
Epoch 10/100, Loss: 1067016847360.0000
Epoch 10/100, Loss: 1612527042560.0000
Epoch 10/100, Loss: 1119000330240.0000
Epoch 10/100, Loss: 1243067318272.0000
Epoch 10/100, Loss: 1278808162304.0000
Epoch 10/100, Loss: 939766317056.0000
Epoch 10/100, Loss: 907543052288.0000
Epoch 10/100, Loss: 882325061632.0000
Epoch 10/100, Loss: 1036119703552.0000
Epoch 10/100, Loss: 848073785344.0000
Epoch 10/100, Loss: 1640812773376.0000
Epoch 10/100, Loss: 1238469967872.0000
Epoch 10/100, Loss: 598987177984.0000
Epoch 10/100, Loss: 1113565298688.0000
Epoch 10/100, Loss: 608884555776.0000
Epoch 10/100, Loss: 106340614144

In [33]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f"MSE: {test_loss.item():.4f}")

MSE: 188492759040.0000


# Saving Model

In [34]:
torch.save(model.state_dict(), 'house_price_model.pth')